# TP3 - VANBELLE Julien

# Clustering de documents

## Imports

In [ ]:
import collections
import os
import string
import sys
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from scipy.spatial.distance import cosine
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
data_path = "../data/txt/"

## Choix de la décennie

In [ ]:
DECADE = '1960'

## Charger tous les  fichiers de la décennie et en créer une liste de textes

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [ ]:
# Exemple de fichiers
files[:5]

In [ ]:
texts = [open(data_path + f).read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

## Vectoriser les documents à l'aide de TF-IDF

In [ ]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Imprimer le vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

### Nombre de clusters

In [ ]:
N_CLUSTERS = 3

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

### Application du clustering à l'aide de la fonction `fit_predict`

In [ ]:
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

## Visualiser les clusters

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

### Générer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

# Word Embeddings : le modèle Word2Vec

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Analyse des bigrams

In [ ]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

In [ ]:
len(bigram_phrases.vocab.keys())

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

In [ ]:
bigram_phrases.vocab[key_]

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Analyse des trigrams

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

### Entrainement de Word2Vec

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=3, # La taille du "contexte", ici 3 mots avant et après le mot observé
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

### Sauvegarde du modèle généré par Word2Vec

In [ ]:
outfile = f"../data/bulletins.model"
model.save(outfile)

### Chargement du modèle

In [ ]:
model = Word2Vec.load("../data/bulletins.model")

## Exploration du modèle

### Similarity(3)

In [ ]:
model.wv.similarity("etablissement", "hopital")

In [ ]:
model.wv.similarity("rue", "boulevard")

In [ ]:
model.wv.similarity("depenses", "francs")

### Most Similar (3)

In [ ]:
model.wv.most_similar("bourgemestre", topn=10)

In [ ]:
model.wv.most_similar("depenses", topn=10)

In [ ]:
model.wv.most_similar("travaux", topn=10)

In [ ]:
print(model.wv.most_similar(positive=['travail', 'communal'], negative=['rue']))

In [ ]:
print(model.wv.most_similar(positive=['travaux', 'communal'], negative=['rue', 'voirie']))

In [ ]:
print(model.wv.most_similar(positive=['payement'], negative=['echevin','bourgemestre','citoyen']))